<a href="https://colab.research.google.com/github/george-marcus/ml-neural-networks/blob/main/MLP_MakeMore.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [24]:
import torch
import torch.nn.functional as F
import matplotlib as plt
from google.colab import drive
import random

%matplotlib inline
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [67]:
words = open('/content/drive/MyDrive/Colab Notebooks/names.txt', 'r').read().splitlines()
chars = sorted(list(set(''.join(words))))
stoi = {s:i+1 for i,s in enumerate(chars)}
stoi['.'] = 0
itos = {i:s for s,i in stoi.items()}
block_size = 3

In [68]:
def build_dataset(words):
  X,Y = [],[]
  for w in words:
    context = [0] * block_size
    for ch in w + '.':
      ix = stoi[ch]
      X.append(context)
      Y.append(ix)
      context = context[1:] + [ix]
  X = torch.tensor(X)
  Y = torch.tensor(Y)
  return X,Y

In [69]:
random.seed(42)
random.shuffle(words)
n1 = int(0.8*len(words))
n2 = int(0.9*len(words))
xtr,ytr = build_dataset(words[:n1])
xdev,ydev = build_dataset(words[n1:n2])
xte,yte = build_dataset(words[n2:])

C = torch.randn((27,10))
W1 = torch.randn((30,200))
b1 = torch.randn(200)
W2 = torch.randn((200,27))
b2 = torch.randn(27)
parameters = [C,W1,b1,W2,b2]

for p in parameters:
  p.requires_grad = True

lre = torch.linspace(-3,0,1000)
lrs = 10**lre
lri = []
lossi = []

In [74]:
for i in range(200000):
  # mini batch construct
  ix = torch.randint(0,xtr.shape[0],(32,))
  # forward pass
  emb = C[xtr[ix]]
  h = torch.tanh(emb.view(-1, 30) @ W1 + b1)
  logits = h @ W2 + b2
  loss = F.cross_entropy(logits,ytr[ix])
  #  backward pass
  for p in parameters:
    p.grad = None
  loss.backward()
  lr = 0.1 if i < 10000 else 0.01

  for p in parameters:
    p.data += -lr * p.grad

In [75]:
emb = C[xdev]
h = torch.tanh(emb.view(emb.shape[0],30) @ W1 + b1)
logits = h @ W2 + b2
loss = F.cross_entropy(logits,ydev)
loss

tensor(2.2382, grad_fn=<NllLossBackward0>)

In [76]:
emb = C[xtr]
h = torch.tanh(emb.view(emb.shape[0],30) @ W1 + b1)
logits = h @ W2 + b2
loss = F.cross_entropy(logits,ytr)
loss

tensor(2.2248, grad_fn=<NllLossBackward0>)

In [77]:
# sample from the model
for _ in range(20):
  out = []
  context = [0] * block_size
  while True:
    emb = C[torch.tensor(context)]
    h = torch.tanh(emb.view(1,-1) @ W1 + b1)
    logits = h @ W2 + b2
    probs = F.softmax(logits, dim=1)
    ix = torch.multinomial(probs, num_samples=1).item()
    context = context[1:] + [ix]
    out.append(ix)
    if ix == 0:
      break
  print(''.join(itos[i] for i in out))

mori.
sehin.
anly.
rusinet.
adair.
jiyah.
grcyelle.
mamdie.
mari.
eylynn.
nia.
teegh.
cambrig.
breirglyn.
zamayda.
tirron.
silliah.
cell.
nai.
prezena.
